# Document Processing with Vertector

This notebook demonstrates document processing capabilities including:
- PDF, DOCX, PPTX, XLSX processing
- Classic vs VLM pipeline selection
- Export to multiple formats
- Table extraction
- OCR configuration

## Setup and Imports

In [1]:
from pathlib import Path
from vertector_data_ingestion import (
    UniversalConverter,
    LocalMpsConfig,
    CloudGpuConfig,
    CloudCpuConfig,
    ExportFormat,
    PipelineType,
    HardwareDetector,
    setup_logging,
)

# Setup logging
setup_logging(log_level="INFO")

2026-01-02 18:37:20 | INFO     | vertector_data_ingestion.monitoring.logger:setup_logging:51 - Logging initialized at INFO level


## Hardware Detection

First, let's detect available hardware to optimize our configuration.

In [2]:
# Detect hardware
hw_info = HardwareDetector.get_device_info()

print("Hardware Information:")
print("=" * 50)
for key, value in hw_info.items():
    print(f"{key}: {value}")

# Get optimized config
hw_config = HardwareDetector.detect()
print(f"\nRecommended device: {hw_config.device_type}")

2026-01-02 18:38:11 | INFO     | vertector_data_ingestion.core.hardware_detector:detect:50 - Detected Apple Silicon with MPS support
2026-01-02 18:38:11 | INFO     | vertector_data_ingestion.core.hardware_detector:detect:50 - Detected Apple Silicon with MPS support


Hardware Information:
device_type: mps
batch_size: 8
use_fp16: False
num_workers: 4
use_mlx: True
platform: darwin
chip: M1

Recommended device: HardwareType.MPS


## Basic Document Conversion

Convert a document using default settings.

In [3]:
# Initialize converter with hardware-optimized config
config = LocalMpsConfig()  # or CloudGpuConfig() or CloudCpuConfig()
converter = UniversalConverter(config)

# Convert a document (replace with your file)
# Note: For audio files (.wav, .mp3), install the 'asr' extra: uv sync --extra asr
doc_path = Path("../test_documents/arxiv_sample.pdf")

2026-01-02 18:38:13 | INFO     | vertector_data_ingestion.core.hardware_detector:detect:50 - Detected Apple Silicon with MPS support
2026-01-02 18:38:13 | INFO     | vertector_data_ingestion.core.hardware_detector:detect:50 - Detected Apple Silicon with MPS support
2026-01-02 18:38:13 | INFO     | vertector_data_ingestion.core.pipeline_router:__init__:55 - Hardware detected: mps
2026-01-02 18:38:13 | INFO     | vertector_data_ingestion.core.universal_converter:__init__:44 - Initialized UniversalConverter on mps
2026-01-02 18:38:13 | INFO     | vertector_data_ingestion.core.universal_converter:_ensure_models_available:67 - Checking model availability...


In [4]:
if doc_path.exists():
    # Use unified convert() method
    doc = converter.convert(doc_path)
    
    print(f"Document: {doc.metadata.source_path.name}")
    print(f"Pages: {doc.metadata.num_pages}")
    print(f"Pipeline: {doc.metadata.pipeline_type}")
    print(f"Processing time: {doc.metadata.processing_time:.2f}s")
else:
    print(f"File not found: {doc_path}")
    print("\nReplace with path to your PDF, DOCX, PPTX, or other document.")

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


2026-01-02 18:38:21 | INFO     | vertector_data_ingestion.core.pipeline_router:determine_pipeline:99 - Using Classic pipeline for PDF with tables: arxiv_sample.pdf
2026-01-02 18:38:21 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:175 - Converting arxiv_sample.pdf with classic pipeline
2026-01-02 18:38:21,570 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-02 18:38:21,665 - INFO - Going to convert document batch...
2026-01-02 18:38:21,666 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 870e160bad93d15722a8ae8d62725e09
2026-01-02 18:38:21,693 - INFO - Loading plugin 'docling_defaults'
2026-01-02 18:38:21,697 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-02 18:38:21,707 - INFO - Loading plugin 'docling_defaults'
2026-01-02 18:38:21,725 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2026-01-02 18:38:27,993 - INFO - Loading plugin 'docling_defa

Document: arxiv_sample.pdf
Pages: 9
Pipeline: classic
Processing time: 51.53s


## Export to Different Formats

In [5]:
if doc_path.exists():
    # Export to Markdown
    markdown = converter.export(doc, ExportFormat.MARKDOWN)
    print("Markdown Output (first 500 chars):")
    print("=" * 50)
    print(markdown[:500])
    print("...\n")
    
    # Save to file - output_dir is automatically managed
    output_path = converter.convert_and_export(
        source=doc_path,
        output_name="document.md",
        format=ExportFormat.MARKDOWN
    )
    print(f"Saved to: {output_path}")

Markdown Output (first 500 chars):
## DocLayNet: A Large Human-Annotated Dataset for Document-Layout Analysis

Birgit Pfitzmann IBM Research Rueschlikon, Switzerland bpf@zurich.ibm.com

Christoph Auer IBM Research Rueschlikon, Switzerland cau@zurich.ibm.com

Ahmed S. Nassar IBM Research

Rueschlikon, Switzerland ahn@zurich.ibm.com

Michele Dolfi IBM Research Rueschlikon, Switzerland dol@zurich.ibm.com

Peter Staar IBM Research Rueschlikon, Switzerland taa@zurich.ibm.com

Figure 1: Four examples of complex page layouts across diff
...



2026-01-02 18:46:28 | INFO     | vertector_data_ingestion.core.pipeline_router:determine_pipeline:99 - Using Classic pipeline for PDF with tables: arxiv_sample.pdf
2026-01-02 18:46:28 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:175 - Converting arxiv_sample.pdf with classic pipeline
2026-01-02 18:46:28,624 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-02 18:46:28,631 - INFO - Going to convert document batch...
2026-01-02 18:46:28,632 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 870e160bad93d15722a8ae8d62725e09
2026-01-02 18:46:28,632 - INFO - Accelerator device: 'mps'
2026-01-02 18:46:30,217 - INFO - Accelerator device: 'mps'
2026-01-02 18:46:31,121 - INFO - Processing document arxiv_sample.pdf
2026-01-02 18:46:51,391 - INFO - Finished converting document arxiv_sample.pdf in 22.77 sec.
2026-01-02 18:46:51 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:194 - Converted a

Saved to: output/document.md


## Pipeline Selection

Compare Classic vs VLM pipelines.

In [6]:
import time

if doc_path.exists():
    print("Classic Pipeline:")
    start = time.time()
    classic_doc = converter.convert(doc_path, use_vlm=False)
    classic_time = time.time() - start
    print(f"  Time: {classic_time:.2f}s")
    
    print("\nVLM Pipeline:")
    start = time.time()
    vlm_doc = converter.convert(doc_path, use_vlm=True)
    vlm_time = time.time() - start
    print(f"  Time: {vlm_time:.2f}s")
    print(f"\nVLM is {vlm_time/classic_time:.2f}x slower (but more accurate)")

2026-01-02 18:48:05 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:175 - Converting arxiv_sample.pdf with classic pipeline
2026-01-02 18:48:05,091 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-02 18:48:05,100 - INFO - Going to convert document batch...
2026-01-02 18:48:05,101 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 870e160bad93d15722a8ae8d62725e09
2026-01-02 18:48:05,102 - INFO - Accelerator device: 'mps'


Classic Pipeline:


2026-01-02 18:48:05,820 - INFO - Accelerator device: 'mps'
2026-01-02 18:48:06,643 - INFO - Processing document arxiv_sample.pdf
2026-01-02 18:48:25,493 - INFO - Finished converting document arxiv_sample.pdf in 20.40 sec.
2026-01-02 18:48:25 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:194 - Converted arxiv_sample.pdf in 20.41s (9 pages, 0.4 pages/sec)
2026-01-02 18:48:25 | INFO     | vertector_data_ingestion.core.pipeline_router:_build_vlm_options:234 - Using Granite-Docling-MLX (258M params, default)
2026-01-02 18:48:25 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:175 - Converting arxiv_sample.pdf with vlm pipeline
2026-01-02 18:48:25,504 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-02 18:48:25,510 - INFO - Going to convert document batch...
2026-01-02 18:48:25,511 - INFO - Initializing pipeline for VlmPipeline with options hash 40c755e91ed89bcff77c660f7fe19c81


  Time: 20.42s

VLM Pipeline:


2026-01-02 18:48:26,727 - INFO - Processing document arxiv_sample.pdf
2026-01-02 18:53:43,198 - INFO - Finished converting document arxiv_sample.pdf in 317.70 sec.
2026-01-02 18:53:43 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:194 - Converted arxiv_sample.pdf in 317.70s (9 pages, 0.0 pages/sec)


  Time: 317.72s

VLM is 15.56x slower (but more accurate)


## Batch Processing

Process multiple documents with the same unified API.

In [7]:
documents_dir = Path("../test_documents/")

if documents_dir.exists():
    pdf_files = list(documents_dir.glob("*.pdf"))[:5]
    
    if pdf_files:
        print(f"Processing {len(pdf_files)} documents...")
        
        # Same convert() method - just pass a list!
        results = converter.convert(pdf_files, parallel=True)
        
        print("\nResults:")
        for doc in results:
            print(f"  {doc.metadata.source_path.name}: {doc.metadata.num_pages} pages")
    else:
        print("No PDF files found")
else:
    print("Create a 'documents/' directory with PDFs to test batch processing")

2026-01-02 18:54:16 | INFO     | vertector_data_ingestion.core.universal_converter:convert_batch:321 - Converting 3 documents in parallel
2026-01-02 18:54:16 | INFO     | vertector_data_ingestion.core.pipeline_router:determine_pipeline:96 - Using VLM pipeline for scanned PDF: ocr_test.pdf


Processing 3 documents...


2026-01-02 18:54:17 | INFO     | vertector_data_ingestion.core.pipeline_router:determine_pipeline:105 - Using Classic pipeline (default) for 2112.13734v2.pdf
2026-01-02 18:54:17 | INFO     | vertector_data_ingestion.core.pipeline_router:_build_vlm_options:234 - Using Granite-Docling-MLX (258M params, default)
2026-01-02 18:54:17 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:175 - Converting 2112.13734v2.pdf with classic pipeline
2026-01-02 18:54:17 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:175 - Converting ocr_test.pdf with vlm pipeline
2026-01-02 18:54:17,681 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-02 18:54:17,712 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-02 18:54:17,758 - INFO - Going to convert document batch...
2026-01-02 18:54:17,809 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 870e160bad93d15722a8ae8d62725e09
2026-01-02 18:54:17,822 -


Results:
  ocr_test.pdf: 1 pages
  2112.13734v2.pdf: 4 pages
  arxiv_sample.pdf: 9 pages


## Summary

This notebook demonstrated:
- Unified `convert()` API for single and batch processing
- Hardware detection
- Pipeline selection
- Export capabilities

Next: Check out `02_audio_transcription.ipynb`